In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
!pip install stellargraph

     |████████████████████████████████| 435 kB 3.2 MB/s 
     |████████████████████████████████| 462 kB 10.0 MB/s 


In [ ]:
def draw_graphs(G,pos_nodes,node_names={},node_size=50,plot_weight=False):
  nx.draw(G,pos_nodes,with_labels=False,node_size=node_size,edge_color="gray",arrowsize=30)
  pos_attrs={}
  for node,coords in pos_nodes.items():
    pos_attrs[node]=(coords[0],coords[1]+0.008)
  nx.draw_networkx_labels(G,pos_attrs,font_family="serif",font_size=30)
  if plot_weight:
    pos_attrs={}
    for node,coords in pos_nodes.items():
      pos_attrs[node]=(coords[0],coords[1]+0.008)
    nx.draw_networkx_labels(G,pos_attrs,font_family="serif",font_size=30)
    edge_labels=dict([((a,b),d["weight"]) for a,b,d in G.edges(data=True)])   
    nx.draw_networkx_edge_labels(G,pos_nodes,edge_labels=edge_labels)
  plt.axis("off")
  axis=plt.gca()
  axis.set_xlim([1.2*x for x in axis.get_xlim()])
  axis.set_ylim([1.2*y for y in axis.get_ylim()]) 


In [ ]:
from stellargraph import datasets
from IPython.display import display,HTML

In [ ]:
dataset=datasets.PROTEINS()
display(HTML(dataset.description))
graphs,graph_labels=dataset.load()

In [ ]:
adj_matrx=[graph.to_adjacency_matrix().A for graph in graphs]
labels=graph_labels.to_numpy()

In [ ]:
metrics=[]
for adj in adj_matrx:
  G=nx.from_numpy_matrix(adj)
  num_edges=G.number_of_edges()
  ff=nx.global_efficiency(G)
  cc=nx.average_clustering(G)
  metrics.append([num_edges,ff,cc])  

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(metrics,labels,test_size=0.2,random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC
mod=SVC()
std=StandardScaler()
X_train_s=std.fit_transform(X_train)
X_test_s=std.fit_transform(X_test)
mod.fit(X_train_s,y_train)
y_train_pred=mod.predict(X_train_s)


In [ ]:
print(classification_report(y_train,y_train_pred))

              precision    recall  f1-score   support

           1       0.74      0.89      0.81       531
           2       0.77      0.54      0.63       359

    accuracy                           0.75       890
   macro avg       0.75      0.71      0.72       890
weighted avg       0.75      0.75      0.74       890



In [ ]:
y_test_pred=mod.predict(X_test)
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           1       0.59      1.00      0.74       132
           2       0.00      0.00      0.00        91

    accuracy                           0.59       223
   macro avg       0.30      0.50      0.37       223
weighted avg       0.35      0.59      0.44       223



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from stellargraph.mapper import PaddedGraphGenerator
generator=PaddedGraphGenerator(graphs=graphs)

In [ ]:
from stellargraph.layer import DeepGraphCNN
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf

In [ ]:
k = 35  # the number of rows for the output tensor
layer_sizes = [32, 32, 32, 1]

dgcnn_model = DeepGraphCNN(
    layer_sizes=layer_sizes,
    activations=["tanh", "tanh", "tanh", "tanh"],
    k=k,
    bias=False,
    generator=generator,
)
x_inp, x_out = dgcnn_model.in_out_tensors()
x_out=tf.keras.layers.GaussianNoise(0.1, seed=None)(x_out)
x_out = Conv1D(filters=16, kernel_size=sum(layer_sizes), strides=sum(layer_sizes))(x_out)
x_out = MaxPool1D(pool_size=2)(x_out)

x_out = Conv1D(filters=32, kernel_size=5, strides=1)(x_out)

x_out = Flatten()(x_out)

x_out = Dense(units=128, activation="relu")(x_out)
x_out = Dropout(rate=0.5)(x_out)

predictions = Dense(units=1, activation="sigmoid")(x_out)
model = Model(inputs=x_inp, outputs=predictions)

model.compile(
    optimizer=Adam(lr=0.0001), loss=binary_crossentropy, metrics=["acc"],
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
from sklearn.model_selection import train_test_split
train_graphs, test_graphs = train_test_split(
    graph_labels, train_size=0.9, test_size=None, stratify=graph_labels,
)
gen = PaddedGraphGenerator(graphs=graphs)

train_gen = gen.flow(
    list(train_graphs.index - 1),
    targets=train_graphs.values,
    batch_size=32,
    symmetric_normalization=False,
)

test_gen = gen.flow(
    list(test_graphs.index - 1),
    targets=test_graphs.values,
    batch_size=1,
    symmetric_normalization=False,
)
epochs = 100
history = model.fit(
    train_gen, epochs=epochs, verbose=1, validation_data=test_gen, shuffle=True,
)